# Matrix Factorization using Gradient Descent

In this notebook I will try to solve the classic problem of matrix factorization using the gradient descent algorithm. In particular, I will factorize a matrix $D$ such that $D \approx CF$. In order to do so, I will use gradient descent in order to train the matrices entries.

$ D =\begin{bmatrix} 
        d_{11} & d_{12} & \dots & d_{1m} \\
        d_{21} &  &  \\
        \vdots &  & \ddots \\
        d_{n1} &  & & d_{nm} \\
        \end{bmatrix}    C =\begin{bmatrix} 
        c_{11} & c_{12} & \dots & c_{1k} \\
        c_{21} &  &  \\
        \vdots &  & \ddots \\
        c_{n1} &  & & c_{nk} \\
        \end{bmatrix}  F =\begin{bmatrix} 
        f_{11} & f_{12} & \dots & f_{1m} \\
        f_{21} &  &  \\
        \vdots &  & \ddots \\
        f_{k1} &  & & f_{km} \\
        \end{bmatrix}$

If we let $F^{i}$ denote the ith column of F, then we have $d_{i,j} \approx C^{Ti} \cdot F^{i}$. 

In [120]:
# Import relevant libraries
import torch                                         # Our main library         
from torch import autograd                           # Needed to compute gradients

To start, let us generate the D matrix. And initializing the C and F matrices.

In [157]:
# Define Dimensions
n = 3
m = 4
k = 2

D = torch.mm(torch.rand(n,k),torch.rand(k,m))
print(D)

tensor([[ 0.0456,  0.1537,  0.0830,  0.0980],
        [ 0.2683,  0.4260,  0.4277,  0.4913],
        [ 0.4779,  0.7156,  0.7563,  0.8673]])


In [158]:
# Since C and F are going to be trained, we set requires_grad=True in order for pytorch to track the operations we do with them

C = torch.rand(n,k, requires_grad=True)
F = torch.rand(k,m, requires_grad=True)
print(C, '\n', F)

tensor([[ 0.9721,  0.2518],
        [ 0.0386,  0.5858],
        [ 0.4206,  0.2610]]) 
 tensor([[ 0.8656,  0.0336,  0.1317,  0.2444],
        [ 0.4919,  0.4990,  0.4563,  0.7689]])


In order to train the weights of C and F, we need to compute the new matrix D_hat as a product between the two matrices. The newly computed matrix will keep track of the gradient w.r.t to the weights of C and F.

In [159]:
D_hat = torch.mm(C,F)
print(D_hat)

tensor([[ 0.9654,  0.1583,  0.2429,  0.4312],
        [ 0.3215,  0.2936,  0.2724,  0.4599],
        [ 0.4925,  0.1443,  0.1745,  0.3034]])


In [160]:
print(D_hat.grad_fn)

Once we compute D_hat, we need to compute our loss function. We will sum the squared differences of the components of D_hat and D.

In [161]:
def criterion(A, B):
    """
    Computes the component by component sum of the squared differences
    """
    return (torch.sum((A-B)**2))

In [162]:
loss = criterion(D, D_hat)
print(loss)

tensor(2.0110)


In [163]:
print(loss.grad_fn)

It is now time to do some backpropagation. The brackpropagation algorothm helps us efficiently compute the gradient, going 'backward'. Let us denote the loss function as $\ell$. Remember thst the loss is a function of C and F. So we have $\ell(C,F)$

In [164]:
loss.backward()

We can now visualize the numerical gradients computed for each input matrix. For instance, $GC_{i,j}(C,F) = \frac{\partial \ell}{\partial C_{i,j}}(C,F) $. Note that the gradient and the loss function are computed for some specific values of C and F. If we change C and F, the loss and its gradient are also going to change.

In [165]:
print(C.grad)

tensor([[ 1.7976,  1.5676],
        [ 0.0270, -0.2698],
        [-0.4419, -1.9538]])


In [166]:
print(F.grad)

tensor([[ 1.8045, -0.4818, -0.1906,  0.1710],
        [ 0.5332, -0.4509, -0.4052, -0.1633]])


Now that we computed the gradient, we can set a learning rate parameted and update the weights. In order to udate the weights, we will simply update them in the direction of greatest descent of the loss function, which is negative the gradient. The learning rate is a parameter that tells us how fast we will update the weights.
Remember to zero out the gradients once this step is finished.

In [167]:
lr = 0.01

# We need to use NO_GRAD to keep the update out of the gradient computation
# Why is that? It boils down to the DYNAMIC GRAPH that PyTorch uses...
with torch.no_grad():
    C -=  lr * C.grad
    F -=  lr * F.grad
    
C.grad.zero_()
F.grad.zero_()

tensor([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]])

In [168]:
print(C, '\n', F)

tensor([[ 0.9541,  0.2361],
        [ 0.0383,  0.5885],
        [ 0.4250,  0.2805]]) 
 tensor([[ 0.8476,  0.0384,  0.1336,  0.2426],
        [ 0.4866,  0.5036,  0.4603,  0.7705]])


If we compare the new C and F matrices, we can see that they have been updated. Also, in the net chinck we are going to check that the loss has indeed decreased with this new weights.

In [169]:
with torch.no_grad():
    D_hat = torch.mm(C,F)
    print(criterion(D, D_hat))

tensor(1.8741)


It has indeed decreased. Let us now build a loop and train the matrices for 100 iterations.

In [170]:
n_iterations = 10000
lr = 0.001
loss_history = []

for i in range(n_iterations):
    
    # Step 0: 0 out gradients
    C.grad.zero_()
    F.grad.zero_()
    # 1st step: forward pass
    D_hat = torch.mm(C,F)
    loss = criterion(D, D_hat)
    # 2nd step: backprop
    loss.backward()
    # 3rd step: parmeters updating
    with torch.no_grad():
        C -=  lr * C.grad
        F -=  lr * F.grad
        loss_history.append(float(loss.detach().numpy()))
        if (i+1)%1000 == 0 or i == 0:
            print('Iteration n. ', '{:d}'.format(i+1), '. Loss: ', '{:.4f}'.format(loss_history[i]))
        

Iteration n.  1 . Loss:  1.8741
Iteration n.  1000 . Loss:  0.0223
Iteration n.  2000 . Loss:  0.0060
Iteration n.  3000 . Loss:  0.0028
Iteration n.  4000 . Loss:  0.0020
Iteration n.  5000 . Loss:  0.0017
Iteration n.  6000 . Loss:  0.0016
Iteration n.  7000 . Loss:  0.0015
Iteration n.  8000 . Loss:  0.0015
Iteration n.  9000 . Loss:  0.0014
Iteration n.  10000 . Loss:  0.0014


Now that the training is complete let's plot D_hat and D, plus their difference.

In [171]:
print(D_hat, '\n',D)

tensor([[ 0.0497,  0.1423,  0.0869,  0.1019],
        [ 0.2768,  0.4008,  0.4364,  0.5000],
        [ 0.4724,  0.7316,  0.7508,  0.8618]]) 
 tensor([[ 0.0456,  0.1537,  0.0830,  0.0980],
        [ 0.2683,  0.4260,  0.4277,  0.4913],
        [ 0.4779,  0.7156,  0.7563,  0.8673]])


In [172]:
print(D_hat-D)

tensor(1.00000e-02 *
       [[ 0.4044, -1.1402,  0.3885,  0.3853],
        [ 0.8437, -2.5201,  0.8680,  0.8727],
        [-0.5418,  1.6038, -0.5506, -0.5531]])
